# Advent of Code Day 19
#### Problem in full can be found here: https://adventofcode.com/2023/day/19

In [245]:
# part 1
# Given an input file where it contains workflows and ratings, these are seperated by an empty line, follow the path
# of the workflows that contain codnitions tos ee if the ratings meet these conditions and get an accepted or rejected
# state and the rating variables to the result if they are accepted.

# Store all the workflows in a dictionary and all ratings in an array
file = open("inputday19test.txt")
workflows = {}
ratings = []
getWorkflows = True
for line in file:
    line = line.strip()
    if line == "":
        getWorkflows = False
    elif getWorkflows:
        i = 0
        key = ""
        while line[i] != '{':
            key += line[i]
            i += 1
        workflows[key] = line[i+1:-1] # append without curly braces
    else:
        ratings.append(line[1:-1]) # append without curly braces
file.close()

# Result where all the accept variables will be added to
result = 0
# Loop through each rating to find if this rating is accepted or rejected
for rating in ratings:
    # Starting position
    current = "in"
    # Get the values for the variables x, m, a, s
    valuesDict = getValuesDict(rating)
    # Loop through the workflows until the rating is accepted or rejected
    while current != "A" and current != "R":
        current = getNextWorkflow(workflows[current], valuesDict)
    # If the rating is accepted, add the variable values to the result
    if current == "A":
        for val in valuesDict.items():
            result += int(val[1])

# print the result
print(result)

# part 2
# Using only the workflows, find the amount of combinations that of the variables x, m, a, s that will result in an accepted state
# given that x, m, a, s can be any integer from 1 to 4000
ranges = {'x': [1, 4000], 'm': [1, 4000], 'a': [1, 4000], 's': [1, 4000]}
combinations = getCombinations("in", ranges)
# print the result
print(combinations)

19114
208574654605400


In [92]:
# A function that calculates the next work flow / state (accepted / rejected) and returns it
def getNextWorkflow(workflow, valuesDict):
    # Get the conditions
    conditions = workflow.split(',')
    # Loop through each condition
    for condition in conditions:
        # If there is a : in the condition, it means there is a condition. If there is no : then there is only a destination
        if ':' in condition:
            # Get the different variables associated in the codition (variable, operator, number to compare variable to, destination)
            variable, operator, num, destination = getWorkflowValues(condition)

            # If the condition is met, return the destination
            if operator == '>' and int(valuesDict[variable]) > int(num):
                return destination
            if operator == '<' and int(valuesDict[variable]) < int(num):
                return destination
        # Return the condition if no ':'
        else:
            return condition

In [84]:
# Function that takes a string as the rating splits it into its variable and value then stores it in a dictionary
# and returns the result
def getValuesDict(rating):
    pairs = rating.split(',')
    ratingValues = [pair.split('=') for pair in pairs]
    valuesDict = {ratingValues[0][0]: ratingValues[0][1], ratingValues[1][0]: ratingValues[1][1], ratingValues[2][0]: ratingValues[2][1], ratingValues[3][0]: ratingValues[3][1]}
    return valuesDict

In [102]:
# Function that takes a string as teh condition and splits into the 4 variables, the variable sign, operator sign,
# number that compares to the variable, and the destination
def getWorkflowValues(condition):
    variable = condition[0]
    operator = condition[1]
    i = 2
    num = ""
    while condition[i] != ':':
        num += condition[i]
        i += 1
    i += 1
    destination = ""
    while i < len(condition):
        destination += condition[i]
        i += 1
    return variable, operator, int(num), destination

In [244]:
import numpy as np

# A function that recursively finds the path and calculates the total
def getCombinations(current, ranges):
    # x: visited nothing (4000, 1)
    # x want to visit (125, 645)
    # x want to visit (100, 875)
    newRanges = ranges.copy()
    conditions = workflows[current].split(',')
    combinations = 0
    
    for condition in conditions:
        if ':' in condition:
            variable, operator, num, destination = getWorkflowValues(condition)
            oldRange = ranges[variable]
            
            # First, update the range to make the condition True
            if operator == '>':
                if num + 1 > ranges[variable][0] and num + 1 < ranges[variable][1]:
                    newRanges[variable][0] = num + 1
            else:
                if num - 1 < ranges[variable][1] and num - 1 > ranges[variable][0]:
                    newRanges[variable][1] = num - 1

            if destination == 'A':
                # Explore the path where the condition is True
                combinations += np.prod(np.array([(newRanges[r][1] - newRanges[r][0] + 1) for r in 'xmas']))
            elif destination == 'R':
                combinations += 0
            else:
                # Explore the path where the condition is True
                combinations += getCombinations(destination, newRanges)
                newRanges[variable] = oldRange  # Revert the range

            # Second, update the range to make the condition False
            if operator == '>':
                if num - 1 > ranges[variable][0] and num - 1 < ranges[variable][1]:
                    newRanges[variable][1] = num - 1
            else:
                if num + 1 < ranges[variable][1] and num + 1 > ranges[variable][0]:
                    newRanges[variable][0] = num + 1

            if destination == 'A':
                # Explore the path where the condition is False
                combinations += np.prod(np.array([(newRanges[r][1] - newRanges[r][0] + 1) for r in 'xmas']))
            elif destination == 'R':
                combinations += 0
            else:
                # Explore the path where the condition is False
                combinations += getCombinations(destination, newRanges)
                newRanges[variable] = oldRange  # Revert the range

        else:
            destination = condition
            if destination == 'A':
                # Explore the path where the condition is True
                combinations += np.prod(np.array([(newRanges[r][1] - newRanges[r][0] + 1) for r in 'xmas']))
            elif destination == 'R':
                combinations += 0
            else:
                # Explore the path where the condition is True
                combinations += getCombinations(destination, newRanges)
                
    return combinations
        